Imports

In [28]:
import re
import html
import pandas as pd
import numpy as np
import unicodedata
import os
from rapidfuzz import fuzz

Définition des listes

In [29]:
TECHNICAL_SKILLS = [
    # Langages
    "python","r","sql","java","c++","scala","javascript","typescript","bash","shell",
    "vba","go","matlab",

    # Data engineering
    "spark","hadoop","airflow","etl","elt","pipeline de données","data pipeline",
    "dataflow","dbt","kafka","bigquery","redshift",
    "snowflake","databricks","data warehouse","entrepôt de données",
    "data lake","lac de données","modélisation de données","data modeling",
    "data integration","data governance","gouvernance des données",

    # BI & Analytics
    "tableau","power bi","looker","data studio","google data studio",
    "domo","microstrategy","superset","qlik","qlikview","qlik sense",
    "excel","google sheets","tableaux de bord","dashboard",
    "visualisation de données","data visualization","business intelligence","bi",

    # Cloud & DevOps
    "aws","amazon web services","azure","gcp","google cloud",
    "linux","unix","git","github","docker","kubernetes","k8s","ci/cd",

    # Data science & ML
    "machine learning","apprentissage automatique","deep learning",
    "apprentissage profond","statistics","statistiques",
    "nlp","traitement du langage naturel","clustering","classification",
    "régression","modélisation prédictive","predictive modeling",
    "ab testing","test a/b","analyse de cohortes",

    # Librairies ML/DS
    "scikit-learn","sklearn","pandas","numpy","scipy","pytorch",
    "tensorflow","keras","matplotlib","seaborn","plotly","d3.js",

    # Data quality
    "data cleaning","nettoyage de données","data validation"
]

TOOLS_LIST = [
    "tableau","power bi","looker","excel","google sheets","aws","azure","gcp",
    "google cloud","bigquery","snowflake","databricks","redshift","mysql",
    "postgresql","postgres","mssql","oracle","jira","confluence","notion",
    "asana","lucidchart","visio","git","github","gitlab","docker",
    "kubernetes","airflow","spark","hadoop","google analytics",
    "matomo","superset","qlik","qlik sense"
]

SOFT_SKILLS = [
    "communication","communication écrite","communication orale",
    "teamwork","travail en équipe","collaboration","leadership",
    "gestion de projet","organisation","autonomie","indépendance",
    "problem solving","résolution de problèmes","esprit critique",
    "critical thinking","analyse","analytical thinking",
    "time management","gestion du temps","adaptabilité",
    "créativité","rigueur","attention aux détails",
    "initiative","prise de décision","curiosité",
    "présentation","presentation skills","gestion des parties prenantes",
    "stakeholder management"
]

EDUCATION_LEVELS = {
    "phd": ["phd","doctorate","doctorat","thèse"],
    "master": ["master","msc","bac+5","m2","magistère"],
    "bachelor": ["bachelor","licence","bsc","ba","bs","bac+3"],
    "associate": ["associate","dut","bts","bac+2"],
    "highschool": ["high school","lycée","baccalauréat"],
    "any": ["diplôme","degree","university degree","college degree"]
}

BENEFITS = [
    "bonus","prime","equity","stock options","actions",
    "health insurance","mutuelle","assurance santé","401k",
    "paid time off","congés payés","jours de repos","sick leave",
    "mental health","santé mentale","remote work","télétravail",
    "flexible schedule","horaires flexibles","titres restaurant",
    "gym","sport","parental leave","congé parental"
]

SENIORITY_LEVELS = {
    "intern": [
        r"\bstagiaire\b",
        r"\ben stage\b",
        r"\bstage \b",
        r"\bintern\b",
        r"\binternship\b",
        r"\balternance\b"
    ],
    "junior": [
        r"\bjunior\b",
        r"entry level",
        r"0-1 ans?", r"1-2 ans?",
        r"débutant"
    ],
    "mid": [
        r"2-4 ans?", r"3-5 ans?",
        r"\bconfirmé\b",
        r"\bintermédiaire\b",
        r"\bmid\b"
    ],
    "senior": [
        r"\bsenior\b",
        r"\biii\b",
        r"5\+? ans?",
        r"6\+? ans?",
        r"expérimenté"
    ],
    "lead": [
        r"\blead\b",
        r"\bprincipal\b",
        r"\bhead\b",
        r"\bmanager\b",
        r"chef de projet"
    ]
}



Fonctions de nettoyage

In [30]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = html.unescape(text)
    text = re.sub(r'<[^>]*>', ' ', text)
    text = re.sub(r'http\S+|www\.\S+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = unicodedata.normalize('NFKC', text)
    text = re.sub(r'[\r\n\t]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

def drop_invalid_rows(df):
    df = df.dropna(subset=["description"])
    df = df[df["description"].str.strip() != ""]
    df = df[df["description"].apply(lambda x: len(x) > 30)]
    return df.reset_index(drop=True)


Fonctions d'extraction

In [31]:
def find_keywords(text, keyword_list):
    found = []
    for k in keyword_list:
        if re.search(r'\b' + re.escape(k) + r'\b', text):
            found.append(k)
    return list(set(found))

def match_category(text, category_dict):
    for cat, patterns in category_dict.items():
        for p in patterns:
            if re.search(p, text.lower()):
                return cat
    return None

def parse_experience(text):
    match = re.search(r'(\d+)\+?\s*(years?|ans?)', text)
    return float(match.group(1)) if match else None


correction des salaires

In [32]:
def fix_salary_column(df, col):
    corrected = []
    for val in df[col]:
        try:
            if pd.isna(val):
                corrected.append(None)
            elif val < 1000:  
                corrected.append(val * 1000)
            else:
                corrected.append(val)
        except:
            corrected.append(None)
    df[col] = corrected
    return df


Pipeline enrichissement

In [33]:
def extract_features(text):
    t = clean_text(text)

    features = {
        "technical_skills": find_keywords(t, TECHNICAL_SKILLS),
        "tools_used": find_keywords(t, TOOLS_LIST),
        "soft_skills": find_keywords(t, SOFT_SKILLS),
        "education_level": match_category(t, EDUCATION_LEVELS),
        "seniority_level": match_category(t, SENIORITY_LEVELS),
        "benefits": find_keywords(t, BENEFITS),
        "experience_years": parse_experience(t)
    }
    return features


In [34]:
TARGET_COLUMNS = [
    "id","title","company","country","location","link","source",
    "date_posted","description","description_sans_html",
    "technical_skills","tools_used","soft_skills",
    "education_level","seniority_level","benefits",
    "eeo_statement","hybrid_policy","visa_sponsorship",
    "tasks","domains","tone_culture","eeo_terms",
    "experience_mentions","salary_value","salary_type",
    "salary_currency","experience_years"
]

for col in TARGET_COLUMNS:
    if col not in df_merged.columns:
        df_merged[col] = ""

df_merged = df_merged[TARGET_COLUMNS]


Chargement fichier scrapper

In [35]:
import csv

def clean_and_enrich_scraped(input_csv, output_csv="master_enriched.csv"):
    print("📥 Chargement du fichier scrappé...")
    
    # 1. Lecture robuste : on lit tout en 'str' pour éviter les erreurs de type (ex: 'ID' dans une colonne chiffre)
    # on_bad_lines='skip' permet de sauter les lignes corrompues (comme la description cassée) sans arrêter le script
    try:
        df = pd.read_csv(
            input_csv, 
            sep="\t", 
            dtype=str, 
            on_bad_lines='skip', 
            engine='python' # Le moteur python est souvent plus permissif
        )
    except Exception as e:
        print(f"⚠️ Erreur lors de la lecture CSV standard : {e}")
        return None

    print(f"📊 Lignes brutes lues : {len(df)}")

    # 2. Suppression des en-têtes répétés (fusion de plusieurs scrapings)
    # On suppose que la première colonne s'appelle 'id' ou '2275' etc. 
    # On regarde si la valeur de la colonne est égale au nom de la colonne
    first_col = df.columns[0]
    df = df[df[first_col] != first_col]
    
    print("🧹 Nettoyage des lignes invalides...")
    df = drop_invalid_rows(df)

    # 3. Conversion des types numériques nécessaires
    # Convertir les colonnes salaires/id en numérique si elles existent, car on a tout lu en 'str'
    cols_to_numeric = ['salary_value', 'salary_min', 'salary_max']
    for col in cols_to_numeric:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    print("✨ Enrichissement Regex...")
    enriched_rows = []

    for _, row in df.iterrows():
        # On s'assure que la description est bien une chaine de caractères
        desc = str(row["description"]) if pd.notna(row["description"]) else ""
        feats = extract_features(desc)
        enriched_rows.append({**row.to_dict(), **feats})

    df_enriched = pd.DataFrame(enriched_rows)

    # Correction des salaires du scraper
    if "salary_value" in df_enriched.columns:
        df_enriched = fix_salary_column(df_enriched, "salary_value")

    if "salary_min" in df_enriched.columns:
        df_enriched = fix_salary_column(df_enriched, "salary_min")

    if "salary_max" in df_enriched.columns:
        df_enriched = fix_salary_column(df_enriched, "salary_max")

    print("💾 Sauvegarde du fichier enrichi...")
    # Utilisation de quotechar='"' et quoting=csv.QUOTE_MINIMAL pour bien gérer les textes
    df_enriched.to_csv(output_csv, sep=";", index=False, quoting=csv.QUOTE_MINIMAL)

    print(f"🎉 Dataset enrichi créé : {output_csv}")
    print(f"👉 Lignes finales : {len(df_enriched)}")

    return df_enriched


In [36]:
import csv
import pandas as pd
import numpy as np
import re
from tqdm import tqdm # Optionnel, pour la barre de chargement

def clean_and_enrich_scraped(input_csv, output_csv="master_enriched.csv"):
    print("📥 Chargement du fichier en mode 'Réparation'...")
    
    # 1. LECTURE ROBUSTE (Modifiée pour ignorer les colonnes cassées)
    # On utilise header=None et quoting=csv.QUOTE_NONE pour lire le fichier brut ligne par ligne
    # sans se soucier des décalages de colonnes causés par les guillemets ou les fusions.
    try:
        df = pd.read_csv(
    input_csv,
    sep=";",
    quotechar='"',
    escapechar="\\",
    dtype=str,
    on_bad_lines="skip",
    engine="python",
    encoding="utf-8"
)

    except Exception as e:
        print(f"⚠️ Erreur critique : {e}")
        return None

    print(f"📊 Lignes brutes lues : {len(df)}")
    
    enriched_rows = []
    
    print("✨ Reconstruction et Enrichissement...")
    
    # On itère sur chaque ligne brute pour reconstruire les colonnes correctement
    # (Cela remplace ton étape 'drop_invalid_rows' de manière plus intelligente)
    
    # Barre de progression si tqdm est installé, sinon itérateur simple
    iterator = tqdm(df.iterrows(), total=len(df)) if 'tqdm' in globals() else df.iterrows()

    for idx, row in iterator:
        # On nettoie la ligne : on enlève les 'nan' et les cases vides
        values = [str(x) for x in row.values if pd.notna(x) and str(x).lower() != 'nan' and str(x).strip() != ""]
        
        if not values:
            continue

        # --- LOGIQUE DE RÉCUPÉRATION ---
        
        # 1. Trouver la Description (C'est toujours le texte le plus long)
        try:
            description = max(values, key=len)
        except ValueError:
            continue 

        # Filtre équivalent à ta fonction 'drop_invalid_rows' : 
        # Si description < 50 chars, c'est probablement un header ou une erreur -> on saute
        if len(description) < 50:
            continue
            
        # 2. Récupérer les infos fixes (ID, Titre, Entreprise) par position
        # Même si les colonnes décalent à la fin, le début (0, 1, 2) reste souvent stable
        job_id = values[0]
        
        # Suppression des en-têtes répétés (ex: si l'ID est "job_id" ou "2275")
        if str(job_id).lower() in ['id', 'job_id', 'job id', 'trackingid']:
            continue
            
        title = values[1] if len(values) > 1 else "Inconnu"
        company = values[2] if len(values) > 2 else "Inconnu"
        
        # 3. Récupérer Date et Lien (qui bougent souvent) via Regex
        # Cherche une date format YYYY-MM-DD
        date = next((s for s in values if re.search(r'\d{4}-\d{2}-\d{2}', s)), None)
        # Cherche un lien http
        link = next((s for s in values if s.startswith('http')), None)

        # --- ENRICHISSEMENT (Ton code original) ---
        # On appelle ta fonction d'extraction sur la description trouvée
        feats = extract_features(description)
        
        # On construit la nouvelle ligne propre
        row_data = {
            "id": job_id,
            "title": title,
            "company": company,
            "date": date,
            "link": link,
            "description": description,
            # On inclut tous les champs générés par extract_features (skills, tools, etc.)
            **feats 
        }
        
        # Note: Les colonnes salaires d'origine sont souvent perdues dans le décalage.
        # On ajoute des placeholders pour que 'fix_salary_column' ne plante pas plus tard
        row_data["salary_value"] = None
        row_data["salary_min"] = None
        row_data["salary_max"] = None

        enriched_rows.append(row_data)

    # Création du DataFrame propre
    df_enriched = pd.DataFrame(enriched_rows)

    # --- NETTOYAGE FINAL (Ton code original) ---
    
    # 3. Conversion et correction des salaires
    # (Même si les valeurs sont souvent None ici vu le parsing, on garde la logique au cas où)
    if "salary_value" in df_enriched.columns:
        df_enriched = fix_salary_column(df_enriched, "salary_value")
    if "salary_min" in df_enriched.columns:
        df_enriched = fix_salary_column(df_enriched, "salary_min")
    if "salary_max" in df_enriched.columns:
        df_enriched = fix_salary_column(df_enriched, "salary_max")

    print("💾 Sauvegarde du fichier enrichi...")
    # Utilisation de QUOTE_ALL pour blinder le fichier de sortie contre les futurs problèmes
    df_enriched.to_csv(output_csv, sep=";", index=False, quoting=csv.QUOTE_ALL)

    print(f"🎉 Dataset enrichi créé : {output_csv}")
    print(f"👉 Lignes finales : {len(df_enriched)}")

    return df_enriched

In [37]:
if __name__ == "__main__":
    clean_and_enrich_scraped("data.csv", "master_enriched.csv")


📥 Chargement du fichier en mode 'Réparation'...
📊 Lignes brutes lues : 14670
✨ Reconstruction et Enrichissement...


100%|██████████| 14670/14670 [02:02<00:00, 119.85it/s]


💾 Sauvegarde du fichier enrichi...
🎉 Dataset enrichi créé : master_enriched.csv
👉 Lignes finales : 14670


Fusionner avec le premier dataset

In [40]:
import pandas as pd
import re

def clean_text(t):
    if pd.isna(t):
        return ""
    t = str(t)
    t = t.replace("\r", "")
    t = t.replace("\n", "\\n")   # OK
    t = t.replace("[", "").replace("]", "")
    return t.strip()


def clean_list_column(t):
    if pd.isna(t):
        return ""
    t = str(t)
    t = t.replace("[", "").replace("]", "")
    t = t.replace("'", "")
    parts = re.split(r"[;,]+", t)
    parts = [p.strip().lower() for p in parts if p.strip()]
    return ";".join(parts)

# Lire tes fichiers bruts (avec ; comme séparateur)
df_site = pd.read_csv("november_17_jobs_updated.csv", sep=";", low_memory=False)
df_new  = pd.read_csv("data.csv", sep=";", low_memory=False)

# Harmoniser la colonne 'link'
for df in (df_site, df_new):
    if "url" in df.columns:
        df["link"] = df["url"]

df_site.drop_duplicates(subset="link", inplace=True)
df_new.drop_duplicates(subset="link", inplace=True)

df_to_add = df_new[~df_new["link"].isin(df_site["link"])]
df_merged = pd.concat([df_site, df_to_add], ignore_index=True)
df_merged.drop_duplicates(subset="link", inplace=True)

# Nettoyer descriptions
df_merged["description"] = df_merged["description"].apply(clean_text)
df_merged["description_sans_html"] = df_merged["description_sans_html"].apply(clean_text)

# Nettoyer skills
df_merged["technical_skills"] = df_merged["technical_skills"].apply(clean_list_column)
df_merged["tools_used"]      = df_merged["tools_used"].apply(clean_list_column)
df_merged["soft_skills"]     = df_merged["soft_skills"].apply(clean_list_column)

# Exporter un CSV propre (séparateur sûr)


df_merged.to_csv(
    "data_universe/data/job_data.csv",
    sep=";",
    index=False,
    encoding="utf-8",
    quoting=csv.QUOTE_ALL,
    lineterminator="\n"
)


print("CSV propre généré ! 🎉")



CSV propre généré ! 🎉
